In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
df=pd.read_csv('../input/titanic/train.csv')

In [ ]:
df_test=pd.read_csv('../input/titanic/test.csv')

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',data=df)

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Sex',data=df,palette='RdBu_r')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Pclass',data=df,palette='rainbow')

In [ ]:
sns.distplot(df['Age'].dropna(),kde=False,color='darkred',bins=40)

In [ ]:
df['Age'].hist(bins=30,color='darkred',alpha=0.3)

In [ ]:
sns.countplot(x='SibSp',data=df)

In [ ]:
df['Fare'].hist(color='green',bins=40,figsize=(8,4))

In [ ]:
import cufflinks as cf
cf.go_offline()

In [ ]:
df['Fare'].iplot(kind='hist',bins=30,color='green')

In [ ]:
plt.figure(figsize=(12, 7))
sns.boxplot(x='Pclass',y='Age',data=df,palette='winter')

**as we can see that in age there is missing value so we create a missing value imputation function by  plotting age range for each class and taking mean for each class to fill missing values **

In [ ]:
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):

        if Pclass == 1:
            return 37

        elif Pclass == 2:
            return 29

        else:
            return 24

    else:
        return Age

In [ ]:
df['Age'] = df[['Age','Pclass']].apply(impute_age,axis=1)

In [ ]:
df.drop('Cabin',axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df['Embarked'].value_counts()

In [ ]:
df['PassengerId'].value_counts()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.info()

**we drop these below 3 columns because they have many distinct values**

In [ ]:
df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
cat=df.select_dtypes(include=['object'])
num=df.select_dtypes(include=['int64','float64'])

In [ ]:
from sklearn import preprocessing 
le = preprocessing.LabelEncoder()
cat=cat.apply(le.fit_transform)

In [ ]:
num.drop('Survived',axis=1,inplace=True)

In [ ]:
num.corr()

In [ ]:
def outlier_capping(x):
    x = x.clip(upper=x.quantile(0.99))
    x = x.clip(lower=x.quantile(0.01))
    return x
num=num.apply(lambda x: outlier_capping(x))

In [ ]:
num.isnull().sum()

In [ ]:
final=pd.concat([num,cat],axis=1)

In [ ]:
final

In [ ]:
X=final.iloc[:,0:7]
y=df['Survived']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, 
                                                    random_state=101)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100,max_depth=3,random_state=123,min_samples_split=2)
clf.fit(X_train,y_train)

In [ ]:
clf.score(X_train,y_train)

In [ ]:
clf.score(X_test,y_test)

In [ ]:
predictions = clf.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(predictions,y_test))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(predictions,y_test))

In [ ]:
df_test

In [ ]:
X.columns

In [ ]:
a=['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex', 'Embarked']

In [ ]:
top_feature=pd.DataFrame()
top_feature=df_test.loc[:,df_test.columns.isin(a)]
#top_feature = top_feature.reindex(sorted(top_feature.columns), axis=1)

In [ ]:
top_feature

In [ ]:
top_feature['Age'] = top_feature[['Age','Pclass']].apply(impute_age,axis=1)

In [ ]:
cat_test=top_feature.select_dtypes(include=['object'])
num_test=top_feature.select_dtypes(include=['int64','float64'])

In [ ]:
cat_test=cat_test.apply(le.fit_transform)

In [ ]:
num_test.isnull().sum()

In [ ]:
def outlier_capping(x):
    x = x.clip(upper=x.quantile(0.99))
    x = x.clip(lower=x.quantile(0.01))
    return x
num_test=num_test.apply(lambda x: outlier_capping(x))

In [ ]:
num_test.fillna(0,inplace=True,axis=0)

In [ ]:
final_test=pd.concat([num_test,cat_test],axis=1)

In [ ]:
prediction_test=clf.predict(final_test)

In [ ]:
prediction_test

In [ ]:
prediction_test=pd.DataFrame(prediction_test)
a=['predicton_survived']
prediction_test.columns=a
output=pd.concat([df_test,prediction_test],axis=1)

In [ ]:
output.to_csv("prediction_results.csv")